In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.common.by import By

In [8]:
def get_jobs(keyword, num_jobs, verbose):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path="C:/Users/marli/Desktop/py4e/YT/ds_salary_proj/chromedriver", options=options)
    driver.set_window_size(1120, 1000)

    #url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    url='https://www.glassdoor.com/Job/united-states-data-scientist-jobs-SRCH_IL.0,13_IN1_KO14,28.htm?clickSource=searchBox'
    driver.get(url)
    jobs = []
    k=0
    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(1)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element(By.XPATH, '//*[@id="MainCol"]/div[1]/ul/li[1]').click()
        except NoSuchElementException:        
        #except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element(By.CSS_SELECTOR,'[alt="Close"]').click()  #clicking to the X.
        except NoSuchElementException:
            pass
        

        try:
            driver.find_element(By.XPATH,'//*[@id="JAModal"]/div/div[2]/span').click()
        except NoSuchElementException:
            pass
        
        #try:
        if k==0:
            driver.find_element(By.XPATH,'//*[@id="onetrust-accept-btn-handler"]').click()            
            k=1
            pass
        
        #Going through each job in this page
        
        for i in range(1,30):

            driver.find_element(By.XPATH,'//*[@id="MainCol"]/div[1]/ul/li['+str(i)+']').click()
            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            #job_button.click()  #You might 
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element(By.XPATH,'//*[@id="JDCol"]/div/article/div/div[1]/div/div/div[1]/div/div[1]/div[1]').text
                    location = driver.find_element(By.XPATH,'//*[@id="JDCol"]/div/article/div/div[1]/div/div/div[1]/div/div[1]/div[3]').text
                    job_title = driver.find_element(By.XPATH,'//*[@id="JDCol"]/div/article/div/div[1]/div/div/div[1]/div/div[1]/div[2]').text
                    job_description = driver.find_element(By.ID,"JobDescriptionContainer").get_attribute('textContent')
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                rating = driver.find_element(By.XPATH,'//*[@id="JDCol"]/div/article/div/div[1]/div/div/div[1]/div[3]/div[1]/div[1]/span').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            
            #Going to the Salary tab
            try:
                if driver.find_element(By.XPATH,'//*[@id="SerpFixedHeader"]/div/div/div[2]').get_attribute('textContent')=="Salary":
                    driver.find_element(By.XPATH,'//*[@id="SerpFixedHeader"]/div/div/div[2]').click
                    try:
                        min_salary=driver.find_element(By.XPATH,"//*[@id='JDCol']/div/article/div/div[2]/div[1]/div[2]/div/div[1]/div[2]/div[2]/div[3]/span[1]").get_attribute('textContent')
                    except NoSuchElementException:
                        min_salary=-1
                    
                    try:    
                        max_salary=driver.find_element(By.XPATH,"//*[@id='JDCol']/div/article/div/div[2]/div[1]/div[2]/div/div[1]/div[2]/div[2]/div[3]/span[2]").get_attribute('textContent')
                    except NoSuchElementException:
                        max_salary=-1
                    
                    try:
                        mean_salary=driver.find_element(By.XPATH,"//*[@id='JDCol']/div/article/div/div[2]/div[1]/div[2]/div/div[1]/div[1]/div[1]").get_attribute('textContent')
                    except NoSuchElementException:
                        mean_salary=-1
                else:
                    min_salary=-1
                    max_salary=-1
                    mean_salary=-1            
                
            except NoSuchElementException:
                pass

            #Going to the Company tab...
            try:
                driver.find_element(By.XPATH,'//*[@id="SerpFixedHeader"]/div/div/div[3]').click
                
                size=-1
                founded=-1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1

                for i in range(1,6):
                    try:
                        if driver.find_element(By.XPATH,'//*[@id="EmpBasicInfo"]/div[1]/div/div['+str(i)+']/span[1]').text=="Size":
                            size = driver.find_element(By.XPATH,'//*[@id="EmpBasicInfo"]/div[1]/div/div['+str(i)+']/span[2]').text
                        if driver.find_element(By.XPATH,'//*[@id="EmpBasicInfo"]/div[1]/div/div['+str(i)+']/span[1]').text=="Founded":
                            founded = driver.find_element(By.XPATH,'//*[@id="EmpBasicInfo"]/div[1]/div/div['+str(i)+']/span[2]').text
                        if driver.find_element(By.XPATH,'//*[@id="EmpBasicInfo"]/div[1]/div/div['+str(i)+']/span[1]').text=="Type":
                            type_of_ownership = driver.find_element(By.XPATH,'//*[@id="EmpBasicInfo"]/div[1]/div/div['+str(i)+']/span[2]').text
                        if driver.find_element(By.XPATH,'//*[@id="EmpBasicInfo"]/div[1]/div/div['+str(i)+']/span[1]').text=="Industry":
                            industry = driver.find_element(By.XPATH,'//*[@id="EmpBasicInfo"]/div[1]/div/div['+str(i)+']/span[2]').text
                        if driver.find_element(By.XPATH,'//*[@id="EmpBasicInfo"]/div[1]/div/div['+str(i)+']/span[1]').text=="Sector":
                            sector = driver.find_element(By.XPATH,'//*[@id="EmpBasicInfo"]/div[1]/div/div['+str(i)+']/span[2]').text
                        if driver.find_element(By.XPATH,'//*[@id="EmpBasicInfo"]/div[1]/div/div['+str(i)+']/span[1]').text=="Revenue":
                            revenue = driver.find_element(By.XPATH,'//*[@id="EmpBasicInfo"]/div[1]/div/div['+str(i)+']/span[2]').text
                    except NoSuchElementException:
                        pass
                    

    

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1

                
            if verbose:
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Job Description" : job_description,
            "Salary_min": min_salary,
            "Salary_max": max_salary,
            "Salary_mean": mean_salary,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element(By.XPATH,'.//*[@id="FooterPageNav"]/div/ul/li[7]/a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [10]:
#This line will open a new chrome window and start the scraping.
df = get_jobs("data scientist", 180, False)
df

C:\Users\marli\AppData\Local\Temp/ipykernel_1704/4075557329.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="C:/Users/marli/Desktop/py4e/YT/ds_salary_proj/chromedriver", options=options)


Progress: 0/180
Progress: 1/180
Progress: 2/180
Progress: 3/180
Progress: 4/180
Progress: 5/180
Progress: 6/180
Progress: 7/180
Progress: 8/180
Progress: 9/180
Progress: 10/180
Progress: 11/180
Progress: 12/180
Progress: 13/180
Progress: 14/180
Progress: 15/180
Progress: 16/180
Progress: 17/180
Progress: 18/180
Progress: 19/180
Progress: 20/180
Progress: 21/180
Progress: 22/180
Progress: 23/180
Progress: 24/180
Progress: 25/180
Progress: 26/180
Progress: 27/180
Progress: 28/180
Progress: 29/180
Progress: 30/180
Progress: 31/180
Progress: 32/180
Progress: 33/180
Progress: 34/180
Progress: 35/180
Progress: 36/180
Progress: 37/180
Progress: 38/180
Progress: 39/180
Progress: 40/180
Progress: 41/180
Progress: 42/180
Progress: 43/180
Progress: 44/180
Progress: 45/180
Progress: 46/180
Progress: 47/180
Progress: 48/180
Progress: 49/180
Progress: 50/180
Progress: 51/180
Progress: 52/180
Progress: 53/180
Progress: 54/180
Progress: 55/180
Progress: 56/180
Progress: 57/180
Progress: 58/180
Progres

,Job Title,Job Description,Salary_min,Salary_max,Salary_mean,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Data Scientist,"Data Scientist QuaEra, a rapidly growing NY/NJ...",$70K,$90K,-1,-1,Aignyte,Remote,-1,-1,-1,-1,-1,-1
1,Data Scientist,"Great Minds, a rapidly growing challenger bran...",-1,-1,-1,3.8,Great Minds\n3.8,Remote,51 to 200 Employees,2007,Nonprofit Organization,K-12 Education,Education,-1
2,Data Scientist or Bioinformatician,Center Information: The Quantitative Biomedica...,$60K,$100K,-1,-1,,Remote,10000+ Employees,1943,Hospital,Health Care Services & Hospitals,Health Care,-1
3,Data Scientist,Job DescriptionACM Analytics is seeking a Data...,$55.00 /hr,$70.00,-1,-1,"ACM Analytics, LLP",Remote,-1,-1,-1,-1,-1,-1
4,Clinical Data Scientist,Job Summary:Data mining of genomics data and c...,$78K,$181K,-1,-1,Baylor Genetics,Remote,201 to 500 Employees,2015,Company - Private,Health Care Services & Hospitals,Health Care,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,Data Scientist,"Data Scientist QuaEra, a rapidly growing NY/NJ...",$70K,$90K,-1,-1,Aignyte,Remote,-1,-1,-1,-1,-1,-1
176,Clinical Data Scientist,Job Summary:Data mining of genomics data and c...,$78K,$181K,-1,-1,Baylor Genetics,Remote,201 to 500 Employees,2015,Company - Private,Health Care Services & Hospitals,Health Care,-1
177,Data Scientist,Employment InformationJob Title: Data Scientis...,$75K,$100K,-1,-1,"GSSA, LLC",Remote,-1,-1,-1,-1,-1,-1
178,Data Scientist,Note: This role is 100% remote.Neal Analytics...,-1,-1,-1,-1,Neal Analytics,Remote,51 to 200 Employees,2011,Company - Private,IT Services,Information Technology,-1


In [11]:
df.to_csv("glassdoor_jobs.csv")